This notebook is for the purpose of running the `distillbert base uncased emotion` model hosted on huggingface. 

In [5]:
import pandas as pd
import json
import torch
import os
from torch.nn.functional import cosine_similarity
from tqdm import tqdm
import numpy as np

In [6]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertTokenizer, BertModel
from transformers import DistilBertModel, DistilBertTokenizer
from sentence_transformers import SentenceTransformer

tokenizer = DistilBertTokenizer.from_pretrained('bhadresh-savani/distilbert-base-uncased-emotion')
model = SentenceTransformer('bhadresh-savani/distilbert-base-uncased-emotion')

No sentence-transformers model found with name bhadresh-savani/distilbert-base-uncased-emotion. Creating a new one with MEAN pooling.


In [7]:
# Load data
clean_ef_data = pd.read_csv('tabulated_cleaned_emotionfiltered_trec.csv')
clean_ef_data = clean_ef_data.drop(['polarity', 'self_ref', 'PRE', 'POST'], axis=1)
print("clean emotion filtered data shape: ", clean_ef_data.shape)
clean_ef_data.head()

clean emotion filtered data shape:  (27981, 2)


,docid,TEXT
0,s_1287_153_9,I mean what the hell bro.
1,s_1287_187_0,"Yeah, crazy isn't it?"
2,s_1287_204_0,No :( sadly it doesn't have everything
3,s_1287_222_4,I'm worried.
4,s_1287_240_1,Better weapons and going against a weaker team...


In [8]:
# Load set of augmented answers
aug_answer_set = pd.read_csv('augmented_answer_sets.csv')
print("aug answer set shape: ", aug_answer_set.shape)
aug_answer_set.sample(4)

aug answer set shape:  (928, 3)


,Question,Severity,Text
155,4,3,I am not finding fulfillment in anything anymore
305,7,4,I am my worst enemy
766,18,2,My appetite is weaker than before
749,17,4,I am too lethargic to do anything


Now that we have answer sets and input text, let's create embeddings.

In [9]:
# Create tokens and vector embeddings for user posts
post_embeddings_df = clean_ef_data.copy()

post_embeddings = post_embeddings_df['TEXT'].to_list()

post_embeddings = model.encode(post_embeddings, device='cuda', show_progress_bar=True, 
                               output_value='sentence_embedding', convert_to_tensor=True)

Batches:   0%|          | 0/875 [00:00<?, ?it/s]

In [6]:
# Initialize an empty dictionary to store the embeddings
bdi_embeddings = {}

# Loop over all 21 BDI questions
for i in tqdm(range(1, 22), total=21):
    # Filter the DataFrame for the current question and severity > 1
    bdi_i_embedding_df = aug_answer_set[(aug_answer_set['Question'] == i) & (aug_answer_set['Severity'] > 1)]
    
    # Get embeddings for the filtered DataFrame
    bdi_i_embeddings = model.encode(
        bdi_i_embedding_df['Text'].to_list(), device='cuda', output_value='sentence_embedding', convert_to_tensor=True
    )
    
    # Store the embeddings in the dictionary
    bdi_embeddings[f"q{i}"] = bdi_i_embeddings

100%|██████████| 21/21 [00:00<00:00, 52.52it/s]


Now that we have embeddings for each post and the associated question, we will rank. 

The rankings will be computed for each question, and are based on the max-similarity 

between a post's embedding and the embedding array of the corresponding question.

In [7]:
cosine_similarity_dict = {}

for i in range(1, 22):

    # Get the correct embeddings list by key
    qa_embeddings = bdi_embeddings[f"q{i}"]

    # Get the max cosine similarity between each post embedding and the answer set for the current BDI question
    qi_cos_similarities = [
        torch.max(cosine_similarity(post_embedding, qa_embeddings)).item()
        for post_embedding in tqdm(post_embeddings, total=len(post_embeddings))
    ]

    # Assign these max cosine similarity rankings to the cosine similarity dictionary
    cosine_similarity_dict[f'q{i}'] = qi_cos_similarities

100%|██████████| 27981/27981 [00:02<00:00, 9574.02it/s]


Now that we have cosine similarities for each question, we need to assign them post ids

In [8]:
# Initialize an empty dictionary to store the rankings for each question
rankings_dict = {}

# Loop over all 21 BDI questions
for i in tqdm(range(1, 22), total=21):
    # Make a copy of the clean_ef_data DataFrame
    q_rankings = clean_ef_data.copy()
    
    # Add a 'score' column to the DataFrame, which contains the cosine similarity scores for the current question
    q_rankings['score'] = cosine_similarity_dict[f'q{i}']
    
    # Sort the DataFrame by the 'score' column in descending order and reset the index
    q_rankings = q_rankings.sort_values('score', ascending=False, ignore_index=True)
    
    # Keep only the top 1000 rows
    q_rankings = q_rankings.head(1000)
    
    # Store the DataFrame in the rankings_dict dictionary with the key as the current question
    rankings_dict[f'q{i}'] = q_rankings

100%|██████████| 21/21 [00:00<00:00, 137.02it/s]


In [9]:
# Format the rankings dict as trec-formatted content before saving to disk.
for i, key in enumerate(rankings_dict.keys()):
    rankings_dict[key]['query'] = f'{i+1}'
    rankings_dict[key]['q0'] = '0'
    rankings_dict[key]['rank'] = range(1, 1001)
    rankings_dict[key]['rank'] = rankings_dict[key]['rank'].astype(str)
    rankings_dict[key]['model'] = 'distilbert-base-uncased-emotion'
    rankings_dict[key] = rankings_dict[key][["query", "q0", "docid", "score", "rank", "model"]]


In [10]:
for key, df in rankings_dict.items():
    df.to_csv(f'distilbert-base-uncased-ranking-outputs/{key}_rankings.tsv', sep='\t', index=False)

In [11]:
# Concatenate all the DataFrames in the dictionary
all_rankings = pd.concat(rankings_dict.values(), ignore_index=True)

# Save the concatenated DataFrame as a TSV file
all_rankings.to_csv('dbue_results.tsv', sep='\t', index=False)

-----------------------------------------------------------------------------------------------------------

Now we have completed the rankings portion. We need to compare rankings against our qrels to get performance metrics.

-----------------------------------------------------------------------------------------------------------------------------

In [12]:
from ranx import Run, Qrels, evaluate
import pandas as pd

In [13]:
# Read the tsv file
distilbert_rankings = pd.read_csv("dbue_results.tsv", sep="\t")
distilbert_rankings['query'] = distilbert_rankings['query'].astype(str)
distilbert_rankings.columns = ["q_id", "q0", "doc_id", "score", "rank", "model"]
distilbert_rankings.head()

,q_id,q0,doc_id,score,rank,model
0,1,0,s_993_781_1,0.997397,1,distilbert-base-uncased-emotion
1,1,0,s_2286_168_4,0.997172,2,distilbert-base-uncased-emotion
2,1,0,s_17_843_10,0.994586,3,distilbert-base-uncased-emotion
3,1,0,s_962_142_0,0.993817,4,distilbert-base-uncased-emotion
4,1,0,s_2953_456_0,0.992738,5,distilbert-base-uncased-emotion


In [14]:
qrels_majority = pd.read_csv('task1/training/t1_training/TRAINING DATA (2023 COLLECTION)/g_qrels_majority_2.csv')
qrels_majority['query'] = qrels_majority['query'].astype(str)
qrels_majority.columns = ["q_id", "q0", "doc_id", "score"]
qrels_majority.head()

,q_id,q0,doc_id,score
0,1,0,s_405_1279_15,1
1,1,0,s_2519_356_0,0
2,1,0,s_2038_51_7,1
3,1,0,s_975_61_2,1
4,1,0,s_577_923_1,1


In [15]:
# Load the ground truth and predicted rankings
qrels = Qrels.from_df(qrels_majority)
run = Run.from_df(distilbert_rankings)

# # Compute the evaluation metrics
results = evaluate(qrels, run, metrics=['r-precision', 'precision@10', 'map', 'ndcg@1000'])

# # Print the results
print(results)

{'r-precision': 0.033224713705138666, 'precision@10': 0.20952380952380953, 'map': 0.016038118447310595, 'ndcg@1000': 0.07169270570364394}
